In [ ]:
%matplotlib notebook

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Test interpolating missing coordinates

## make an incomplete grid

In [ ]:
# make an incomplete grid
x = np.arange(4).astype(float)
y = np.arange(4).astype(float)
xx, yy = np.meshgrid(x, y, indexing='ij')
zz = xx + yy

x = xx.reshape(-1)
y = yy.reshape(-1)
z = zz.reshape(-1)

x[10:] = np.nan
y[10:] = np.nan
z[10:] = np.nan

xx = x.reshape(xx.shape)
yy = y.reshape(yy.shape)

xx, yy

## find missing values in x and y

In [ ]:
## find missing values in the x axis
pd.DataFrame(xx).interpolate(axis=1)

In [ ]:
## find missing values in the y axis
pd.DataFrame(yy).interpolate(axis=0).values

## missing values in the other 'direction'

In [ ]:
## make an incomplete grid the other 'direction'
x = np.arange(4).astype(float)
y = np.arange(4).astype(float)
xx, yy = np.meshgrid(x, y, indexing='ij')
zz = xx + yy

x = xx.T.reshape(-1)
y = yy.T.reshape(-1)
z = zz.T.reshape(-1)

x[10:] = np.nan
y[10:] = np.nan
z[10:] = np.nan

xx = x.reshape(xx.shape).T
yy = y.reshape(yy.shape).T

xx, yy

## finding the missing values

In [ ]:
## find missing values in the x axis
pd.DataFrame(xx).interpolate(axis=1)

In [ ]:
## find missing values in the y axis
pd.DataFrame(yy).interpolate(axis=0).values

# making vertices from grid centers

In [ ]:
### make vertices from grid centers (for pcolormesh)
x = np.arange(4).astype(float)
y = np.arange(4).astype(float)
xx, yy = np.meshgrid(x, y, indexing='ij')
zz = xx + yy

xx

In [ ]:
def centers2edges_2d(centers):
    """
    Given a 2d array of coordinates, return the array of bounding vertices for
    the mesh that is defined by the coordinates.
    This is useful for methods like pcolor(mesh).
    Done by very simple linear interpolation.
    
    To illustrate: if x are the centers, we return the inferred o's:

    o   o   o   o
      x---x---x
    o | o | o | o
      x---x---x
    o   o   o   o
    """
    shp = centers.shape
    edges = np.zeros((shp[0]+1, shp[1]+1))
    
    # the central vertices are easy -- follow just from the means of the neighboring centers
    center = (centers[1:,1:] + centers[:-1,:-1] + centers[:-1,1:] + centers[1:,:-1])/4.
    edges[1:-1, 1:-1] = center
    
    # for the outer edges we just make the vertices such that the points are pretty much in the center
    # first average over neighbor centers to get the 'vertical' right
    _left = (centers[0,1:] + centers[0,:-1])/2.
    # then extrapolate to the left of the centers
    left = 2 * _left - center[0,:]
    edges[0, 1:-1] = left
    
    # and same for the other three sides
    _right = (centers[-1,1:] + centers[-1,:-1])/2.
    right = 2 * _right - center[-1,:]
    edges[-1, 1:-1] = right
    
    _top = (centers[1:,0] + centers[:-1,0])/2.
    top = 2 * _top - center[:,0]
    edges[1:-1, 0] = top
    
    _bottom = (centers[1:,-1] + centers[:-1,-1])/2.
    bottom = 2 * _bottom - center[:,-1]
    edges[1:-1, -1] = bottom
    
    # only thing remaining now is the corners of the grid.
    # this is a bit simplistic, but will be fine for now.
    # (mirror the vertex (1,1) of the diagonal at the outermost center)
    edges[0,0] = 2 * centers[0,0] - edges[1,1]
    edges[0,-1] = 2 * centers[0,-1] - edges[1,-2]
    edges[-1,0] = 2 * centers[-1,0] - edges[-2,1]
    edges[-1,-1] = 2 * centers[-1,-1] - edges[-2,-2]
    
    return edges

## Testing

In [ ]:
### make vertices from grid centers (for pcolormesh)
x = np.arange(5, 15).astype(float)
y = np.arange(-4, 5).astype(float)


xx, yy = np.meshgrid(x, y, indexing='ij')
zz = np.random.random(xx.shape)

xx1d = xx.reshape(-1)
yy1d = yy.reshape(-1)
xx1d += (np.random.rand(xx1d.size)* 0.5 - 0.25)
yy1d += (np.random.rand(yy1d.size)* 0.5 - 0.25)
xx, yy = xx1d.reshape(xx.shape), yy1d.reshape(yy.shape)

xx2 = centers2edges_2d(xx)
yy2 = centers2edges_2d(yy)

fig, ax = plt.subplots(1,1)
ax.pcolormesh(xx2, yy2, zz)
ax.plot(xx1d, yy1d, 'o', ms=4, mew=1, mec='k', mfc='w')